In [ ]:
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
from nltk.corpus import stopwords
import nltk
import re
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import urllib3, socket
from urllib3.connection import HTTPConnection

In [ ]:
def syllable_and_count(word ,syllable_count,syllable_word_count,complex_word):
    count = 0
    syllable = []
    vowels = "aeiou"
    for i in word:
      if i in vowels:
        syllable.append(i)
        count+=1

    if (word[-2:]=="ed") or (word[-2:]=='es'):
      count=-1
    if count<0:
      count = 0
    if count>0:
      syllable_count+=len(set(syllable))
      if len(set(syllable))>=2:
        complex_word+=1
    syllable_word_count+=1

    return (complex_word,syllable_count,syllable_word_count)


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

en_stopwords = set(stopwords.words('english'))

files_stopwords = ["Stopwords/StopWords_Auditor.txt",
                   "Stopwords/StopWords_Currencies.txt",
                   "Stopwords/StopWords_DatesandNumbers.txt",
                   "Stopwords/StopWords_Generic.txt",
                   "Stopwords/StopWords_GenericLong.txt",
                   "Stopwords/StopWords_Geographic.txt",
                   "Stopwords/StopWords_Names.txt"]

for file in files_stopwords:
    try:
        w = open("/content/{filename}".format(filename=file), 'r', encoding='utf-8').read().strip().lower()
    except UnicodeDecodeError:
        w = open("/content/{filename}".format(filename=file), 'r', encoding='latin-1').read().strip().lower()
    en_stopwords.update(w.split())

extra_stopwords = ["./td-block-span12",
                   "/.content",
                   "./td-related-span4",
                   "./block",
                   "./block1",
                   "/.td-pb-row",
                   "/.td-container",
                   "/.post"]

en_stopwords.update(extra_stopwords)

print(len(en_stopwords))

12872


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
en_stopwords

{'shirey',
 'stahl',
 'davidson',
 'larue',
 'cerda',
 'antony',
 'deloitte',
 'sims',
 'waits',
 'clifford',
 'dew',
 'garver',
 'shae',
 'santana',
 'timmerman',
 'joeann',
 'hann',
 'huang',
 'beatrice',
 'cristy',
 'thirty',
 'lon',
 'katina',
 'tenorio',
 'cambridge',
 'maciel',
 'stouffer',
 'tommie',
 'behr',
 'novella',
 'lemus',
 'edyth',
 'bannister',
 'vanetta',
 'goebel',
 'groce',
 'wild',
 'gibson',
 'hafer',
 'hawthorne',
 'camila',
 'idell',
 'suanne',
 'marisela',
 'natasha',
 'janessa',
 'jeannine',
 'tabitha',
 'karissa',
 'englert',
 'weldon',
 'jess',
 'elfrieda',
 'han',
 'schmid',
 'jeter',
 'deberry',
 'lorenz',
 'sexton',
 'parrott',
 'peabody',
 "she's",
 'lisabeth',
 'pamula',
 'banister',
 'yourself',
 'cherie',
 'lory',
 'ellan',
 'turk',
 'vatu',
 'shapiro',
 'fishman',
 'you',
 'petry',
 'kareem',
 'adell',
 'caroyln',
 'juli',
 'jocelyn',
 'georgine',
 'chaffin',
 'hisako',
 'estes',
 'mapes',
 'tanja',
 'dorsey',
 'garrick',
 'vesta',
 'alley',
 'luck',

#Negative words

In [ ]:
# Try reading the file with UTF-8 encoding
try:
    negative_words = open("/content/Stopwords/{filename}".format(filename="negative-words.txt"), 'r', encoding='utf-8').read().strip().lower()
except UnicodeDecodeError:
    # If UTF-8 decoding fails, try with Latin-1 encoding
    negative_words = open("/content/Stopwords/{filename}".format(filename="negative-words.txt"), 'r', encoding='latin-1').read().strip().lower()

negative_words = set(negative_words.split())


#Positive words

In [ ]:
positive_words=open("/content/Stopwords/{filename}".format(filename="positive-words.txt"),'r').read().strip().lower()
positive_words=set(positive_words.split())
# print(positive_words)
# print(len(positive_words))

In [ ]:
# declearing  arrays to store outputs
POSITIVE_SCORE=[]
NEGATIVE_SCORE=[]
POLARITY_SCORE=[]
SUBJECTIVITY_SCORE=[]
AVG_SENTENCE_LENGTH=[]
PercentageOfComplexWords=[]
FogIndex=[]
AVG_NUMBER_OF_WORDS_PER_SENTENCE=[]
COMPLEX_WORD_COUNT=[]
WORD_COUNT=[]
PERSONAL_PRONOUNS=[]
SYLLABLE_PER_WORD=[]
AVG_WORD_LENGTH=[]


In [ ]:
%pip install pandas youtube-transcript-api nltk openpyxl

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

# Read the Excel file
df = pd.read_excel('/content/Input.xlsx')

# Assuming the YouTube links are in a column named 'YouTube Link'
URL = df['URL'].tolist()

In [ ]:
def extract_video_ids(youtube_links):
    video_ids = []
    for link in youtube_links:
        try:
            video_id = link.split('?v=')[1]
            video_ids.append(video_id)
        except IndexError:
            print(f"Invalid YouTube link: {link}")
    return video_ids

# Extract video IDs
video_ids = extract_video_ids(URL)

# Function to extract transcripts using video IDs
def extract_transcripts(video_ids):
    transcripts = []
    count = 0
    for video_id in video_ids:
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['en', 'en-IN', 'ja', 'hi', 'de'])
            transcript = ' '.join([t['text'] for t in transcript_list])
            transcripts.append(transcript)
        except Exception as e:
            print(f"Error processing video ID {video_id}: {e}")
            count+=1
            transcript = ' '.join("")
            transcripts.append(transcript)
    return transcripts, count

# Extract transcripts
extracted_transcripts, count = extract_transcripts(video_ids)

Error processing video ID wXbRW4-kXqM: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=wXbRW4-kXqM! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error processing video ID 9tOtIDjN6_k: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=9tOtIDjN6_k! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-trans

In [ ]:
url_no=0
print("iterating through each url in list")
#iterating through each url in list
for cleaned_text in extracted_transcripts:
  url_no += 1
  # for cleaned_text in text_content:

  if str(cleaned_text) != '': #cleaning request (text_content) if it has any nan value

    nlp = spacy.load('en_core_web_sm')
    # 1. Create an NLP document with Spacy:
    doc = nlp(cleaned_text)
    input_string=str(doc).lower().strip().split()
    # print("doc=--------->>>>\n",doc)

    input_string= str(doc)
    input_string=input_string.lower().split()
    tokens_without_sw = [word for word in input_string if not word in en_stopwords]


    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(str(doc))

    all_words=0
    negative_word_count = 0
    positive_word_count = 0
    complex_word,syllable_count,syllable_word_count =0,0,0
    number_of_sentences=len(list(doc.sents))

    for i in tokens_without_sw:
      all_words+=1 # word counter


      if i in negative_words: # negative word pass
        negative_word_count+=1 # negative word counter

      if i in positive_words: # positive word pass
        positive_word_count+=1 # positive word counter

      complex_word,syllable_count,syllable_word_count=syllable_and_count(i,syllable_count,syllable_word_count,complex_word) #calling syllable_and_count function
    if all_words==0:
      print("*********** All Words are zero ************")

      #setting all valuse to NAN if link responce is NAN
      POSITIVE_SCORE.append("0")
      NEGATIVE_SCORE.append("0")
      POLARITY_SCORE.append("0")
      SUBJECTIVITY_SCORE.append("0")
      AVG_SENTENCE_LENGTH.append("0")
      PercentageOfComplexWords.append("0")
      FogIndex.append("0")
      AVG_NUMBER_OF_WORDS_PER_SENTENCE.append("0")
      COMPLEX_WORD_COUNT.append("0")
      WORD_COUNT.append("0")
      SYLLABLE_PER_WORD.append("0")
      PERSONAL_PRONOUNS.append("0")
      AVG_WORD_LENGTH.append("0")
      continue


    polarity_score = (positive_word_count - negative_word_count) / ((positive_word_count + negative_word_count) + 0.000001) # calculating plock
    subjectivity_score = (positive_word_count - negative_word_count) / (all_words + 0.000001) # calculating subjectivity_score

    Average_Sentence_Length = all_words / number_of_sentences
    Percentage_of_Complex_words = complex_word / all_words
    Fog_Index = 0.4 * (Average_Sentence_Length + Percentage_of_Complex_words)


    #appending calculation to the specific list Respectively
    POSITIVE_SCORE.append(positive_word_count)
    NEGATIVE_SCORE.append(negative_word_count)
    POLARITY_SCORE.append(polarity_score)
    SUBJECTIVITY_SCORE.append(subjectivity_score)
    AVG_SENTENCE_LENGTH.append(Average_Sentence_Length)
    PercentageOfComplexWords.append(Percentage_of_Complex_words)
    FogIndex.append(Fog_Index)
    AVG_NUMBER_OF_WORDS_PER_SENTENCE.append(len(doc)/Average_Sentence_Length)
    COMPLEX_WORD_COUNT.append(complex_word)
    WORD_COUNT.append(all_words)
    SYLLABLE_PER_WORD.append(syllable_count/syllable_word_count)
    PERSONAL_PRONOUNS.append(len(pronouns))
    AVG_WORD_LENGTH.append(len(doc)/all_words)

  else:
    print("*********** NaN found ************")

    #setting all valuse to NAN if link responce is NAN
    POSITIVE_SCORE.append("nan")
    NEGATIVE_SCORE.append("nan")
    POLARITY_SCORE.append("nan")
    SUBJECTIVITY_SCORE.append("nan")
    AVG_SENTENCE_LENGTH.append("nan")
    PercentageOfComplexWords.append("nan")
    FogIndex.append("nan")
    AVG_NUMBER_OF_WORDS_PER_SENTENCE.append("nan")
    COMPLEX_WORD_COUNT.append("nan")
    WORD_COUNT.append("nan")
    SYLLABLE_PER_WORD.append("nan")
    PERSONAL_PRONOUNS.append("nan")
    AVG_WORD_LENGTH.append("nan")


  print(f"completed URL == {url_no}")


iterating through each url in list
*********** NaN found ************
completed URL == 1
completed URL == 2
completed URL == 3
completed URL == 4
completed URL == 5
*********** NaN found ************
completed URL == 6
completed URL == 7
completed URL == 8
completed URL == 9
completed URL == 10
completed URL == 11
*********** NaN found ************
completed URL == 12
completed URL == 13
completed URL == 14
completed URL == 15
completed URL == 16
completed URL == 17
completed URL == 18
completed URL == 19
completed URL == 20
completed URL == 21
completed URL == 22
completed URL == 23
completed URL == 24
completed URL == 25
completed URL == 26
completed URL == 27
completed URL == 28
completed URL == 29
completed URL == 30
completed URL == 31
completed URL == 32
completed URL == 33
*********** NaN found ************
completed URL == 34
completed URL == 35
completed URL == 36
completed URL == 37
completed URL == 38
completed URL == 39
completed URL == 40
completed URL == 41
completed URL 

In [ ]:
len(POSITIVE_SCORE)

63

In [ ]:
import pandas as pd

# creating cata frame
df = pd.DataFrame(columns = ['URL' ,  'POSITIVE SCORE' , 'NEGATIVE SCORE' ,'POLARITY SCORE' ,'SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])

# Adding values to dataframe
for i in range (0,61):
# for i in range (394,490):

  list_1 = [
            URL[i],
            POSITIVE_SCORE[i],
            NEGATIVE_SCORE[i],
            POLARITY_SCORE[i],
            SUBJECTIVITY_SCORE[i],
            AVG_SENTENCE_LENGTH[i],
            PercentageOfComplexWords[i],
            FogIndex[i],
            AVG_NUMBER_OF_WORDS_PER_SENTENCE[i],
            COMPLEX_WORD_COUNT[i],
            WORD_COUNT[i],
            PERSONAL_PRONOUNS[i],
            SYLLABLE_PER_WORD[i],
            AVG_WORD_LENGTH[i]
            ]
  df.loc[len(df)] = list_1

#setting file name
file_name = 'OutPut_1.xlsx'

# saving the excel
df.to_excel(file_name,index=False)
# display
display(df)

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,https://www.youtube.com/watch?v=wXbRW4-kXqM,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,https://www.youtube.com/watch?v=iL3CLJVaG9U,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,https://www.youtube.com/watch?v=eug1sbP5Y-g,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,https://www.youtube.com/watch?v=3OBFc0C9TbM,5,19,-0.583333,-0.092105,8.0,0.605263,3.442105,55.0,92,152,7,1.638158,2.894737
4,https://www.youtube.com/watch?v=EOyYUnf8MT0,5,28,-0.69697,-0.093496,22.363636,0.520325,9.153585,36.711382,128,246,23,1.48374,3.337398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,https://www.youtube.com/watch?v=ytPbO_lgwAs,14,31,-0.377778,-0.05136,41.375,0.586103,16.784441,24.725076,194,331,21,1.55287,3.090634
57,https://www.youtube.com/watch?v=s2SVe7aDoVQ,1,4,-0.6,-0.022556,4.15625,0.548872,1.882049,93.593985,73,133,5,1.526316,2.924812
58,https://www.youtube.com/watch?v=WFcLfDSQuRc,0,8,-1.0,-0.029963,66.75,0.464419,26.885768,4.299625,124,267,0,1.558052,1.074906
59,https://www.youtube.com/watch?v=J3nJN6jD2BI,1,2,-0.333333,-0.033333,3.75,0.6,1.74,20.8,18,30,0,1.633333,2.6
